# Construcción de modelos de aprendizaje

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [61]:
!ls

drive	  sample_data		     spark-3.1.1-bin-hadoop3.2.tgz
lr_model  spark-3.1.1-bin-hadoop3.2


In [62]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
path_file = '/content/drive/MyDrive/BigData_Maestria/Clase/cars.csv'

In [7]:
df = spark.read.csv(path_file, header=True, sep=";", inferSchema=True)
df.show(5)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows



In [8]:
print("Número de registros: " + str(df.count()))
print("Número de columnas: " + str(len(df.columns)))

Número de registros: 406
Número de columnas: 9


In [9]:
df.describe()

summary,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
count,406,406,406,406,406,406,406,406,406
mean,null,23.051231527093602,5.475369458128079,194.7795566502463,103.5295566502463,2979.4138,15.519704433497521,75.92118226600985,null
stddev,null,8.4017773522706,1.712159631548529,104.92245837948867,40.52065912106347,847.0043282393513,2.8033588163425462,3.7487373454558743,null
min,AMC Ambassador Br...,0.0,3,68.0,0.0,1613,8.0,70,Europe
max,Volvo Diesel,46.6,8,455.0,230.0,5140,24.8,82,US


In [10]:
df.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: decimal(4,0) (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)



### Limpieza de datos

El proceso de limpieza de datos depende fuertemente de las características y estado de la muestra con la cual se estará trabajando. En ocasiones, la preparación de los datos requerirá varias etapas de limpieza, transformación, eliminación de ruido, etc. en los datos, mientras que en otras ocasiones dicha preparación constará de etapas muy básicas.

In [11]:
#Se eliminan registros con valores nulos
df_clean = df.dropna()

#Se eliminan columnas con valores nulos
df_clean = df_clean.na.drop()

#Se eliminan registros duplicados
df_clean = df_clean.dropDuplicates()

In [12]:
df_clean.describe()

summary,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
count,406,406,406,406,406,406,406,406,406
mean,null,23.051231527093602,5.475369458128079,194.7795566502463,103.5295566502463,2979.4138,15.519704433497537,75.92118226600985,null
stddev,null,8.401777352270594,1.7121596315485297,104.92245837948875,40.52065912106347,847.0043282393509,2.8033588163425462,3.7487373454558783,null
min,AMC Ambassador Br...,0.0,3,68.0,0.0,1613,8.0,70,Europe
max,Volvo Diesel,46.6,8,455.0,230.0,5140,24.8,82,US


#### Planes de transformación

Es importante tomar en cuenta que al momento de implementar etapas de procesamiento en PySpark, las transformaciones no se lanzan hasta que se genera un disparador. Para ello, es fundamental crear un plan de transformación eficiente para minimizar el costo de las operaciones "wide".
En este ejemplo se ordenan los registros del Dataframe, dónde el ordenamiento es una transformación "wide". Para visualizar el plan que aplica Spark en la transformación, se puede usar el comando "explain" (se lee desde arriba hacia abajo, dónde el tope es el resultado final, mientras que lo más anidado es la fuente de datos). Observemos que el plan indica "Sort -> exchange -> FileScan", lo que indica que es una "wide transformation", ya que se tienen que comparar registros de las diferentes particiones

In [13]:
#se ordenan los registros del dataframe a partir del origen
df_clean_sort = df_clean.orderBy('origin')
#se imprime el plan de transformaciones
df_clean_sort.explain()
df_clean_sort.take(3)


== Physical Plan ==
*(3) Sort [origin#24 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(origin#24 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#139]
   +- *(2) HashAggregate(keys=[Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16], functions=[])
      +- Exchange hashpartitioning(Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16, 200), ENSURE_REQUIREMENTS, [id=#135]
         +- *(1) HashAggregate(keys=[Cylinders#18, knownfloatingpointnormalized(normalizenanandzero(Horsepower#20)) AS Horsepower#20, Origin#24, Weight#21, Model#23, knownfloatingpointnormalized(normalizenanandzero(MPG#17)) AS MPG#17, knownfloatingpointnormalized(normalizenanandzero(Acceleration#22)) AS Acceleration#22, knownfloatingpointnormalized(normalizenanandzero(Displacement#19)) AS Displacement#19, Car#16], functions=[])
            +- *(1) Filter AtLeastNNulls(n, Car#16,MPG#17,Cylinders

[Row(Car='Volvo 144ea', MPG=19.0, Cylinders=4, Displacement=121.0, Horsepower=112.0, Weight=Decimal('2868'), Acceleration=15.5, Model=73, Origin='Europe'),
 Row(Car='Fiat Strada Custom', MPG=37.3, Cylinders=4, Displacement=91.0, Horsepower=69.0, Weight=Decimal('2130'), Acceleration=14.7, Model=79, Origin='Europe'),
 Row(Car='Volkswagen Rabbit l', MPG=36.0, Cylinders=4, Displacement=105.0, Horsepower=74.0, Weight=Decimal('1980'), Acceleration=15.3, Model=82, Origin='Europe')]

Cuando en una transformación se involucra una "wide transformation", se forza a una mezcla de datos entre las diferentes particiones existentes en el cluster. Por default, Spark genera 200 particiones de mezcla, pero ello se puede controlar. Para ello se puede invocar a "spark.conf.set", casí como se muestra en el siguiente ejemplo:

In [14]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

df_clean_sort = df_clean.orderBy('origin')
#se imprime el plan de transformaciones
df_clean_sort.explain()
df_clean_sort.take(3)

== Physical Plan ==
*(3) Sort [origin#24 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(origin#24 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [id=#199]
   +- *(2) HashAggregate(keys=[Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16], functions=[])
      +- Exchange hashpartitioning(Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16, 5), ENSURE_REQUIREMENTS, [id=#195]
         +- *(1) HashAggregate(keys=[Cylinders#18, knownfloatingpointnormalized(normalizenanandzero(Horsepower#20)) AS Horsepower#20, Origin#24, Weight#21, Model#23, knownfloatingpointnormalized(normalizenanandzero(MPG#17)) AS MPG#17, knownfloatingpointnormalized(normalizenanandzero(Acceleration#22)) AS Acceleration#22, knownfloatingpointnormalized(normalizenanandzero(Displacement#19)) AS Displacement#19, Car#16], functions=[])
            +- *(1) Filter AtLeastNNulls(n, Car#16,MPG#17,Cylinders#18,

[Row(Car='Fiat 124B', MPG=30.0, Cylinders=4, Displacement=88.0, Horsepower=76.0, Weight=Decimal('2065'), Acceleration=14.5, Model=71, Origin='Europe'),
 Row(Car='Fiat 128', MPG=29.0, Cylinders=4, Displacement=68.0, Horsepower=49.0, Weight=Decimal('1867'), Acceleration=19.5, Model=73, Origin='Europe'),
 Row(Car='Volkswagen 411 (sw)', MPG=22.0, Cylinders=4, Displacement=121.0, Horsepower=76.0, Weight=Decimal('2511'), Acceleration=18.0, Model=72, Origin='Europe')]

In [15]:
df_clean_sort.show(10)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|           Fiat 124B|30.0|        4|        88.0|      76.0|  2065|        14.5|   71|Europe|
|            Fiat 128|24.0|        4|        90.0|      75.0|  2108|        15.5|   74|Europe|
|Citroen DS-21 Pallas| 0.0|        4|       133.0|     115.0|  3090|        17.5|   70|Europe|
|Volkswagen 1131 D...|26.0|        4|        97.0|      46.0|  1835|        20.5|   70|Europe|
| Volkswagen 411 (sw)|22.0|        4|       121.0|      76.0|  2511|        18.0|   72|Europe|
|            Fiat 128|29.0|        4|        68.0|      49.0|  1867|        19.5|   73|Europe|
|          Opel Manta|24.0|        4|       116.0|      75.0|  2158|        15.5|   73|Europe|
|         Volvo 144ea|19.0|        4|       121.0|

Definido un plan lógico de transformaciones, Spark sabe como aplicar dicho plan a las particiones que se estén trabajando, con lo que si se vuelva a aplicar dicho plan a los mismos datos de entrada, se obtendrá el mismo resultado (programación funcional, donde las mismas entradas dan las mismas salidas siempre y cuando las transformaciones sobre esos datos permanezcan constantes).

#### Manipulación a través de SQL

Se pueden definir las transformaciones a través del lenguaje de consulta SQL. Cuando se definen las transformaciones a través de SQL, Spark compilará el plan de acción antes de su ejecución. Para ello, primero se debe de registrar un Dataframe como una tabla o vista (view), con lo cual se pueden armar las consultas SQL puro. En principio, no existe diferencia entre consultas SQL y su equivalente en operadores nativos de Dataframes Spark.

In [16]:
df_clean_sort.createOrReplaceTempView("cars")

Ahora ya se pueden definir consultas en SQL, dónde las salidas serán Dataframes de Spark. A continuación se muestran dos planes de acción, una definida con SQL y la otra a través de comandos nativos de Dataframes de Spark, y se usa explain() para visualizar el plan de acción:

In [17]:
query = """SELECT Origin, count(1) FROM cars GROUP BY Origin"""
df_sql = spark.sql(query)
df_sql.explain()

df_nosql = df_clean_sort.groupBy('Origin').count()
df_nosql.explain()

df_sql.show(5)
df_nosql.show(5)

== Physical Plan ==
*(3) HashAggregate(keys=[Origin#24], functions=[count(1)])
+- Exchange hashpartitioning(Origin#24, 5), ENSURE_REQUIREMENTS, [id=#294]
   +- *(2) HashAggregate(keys=[Origin#24], functions=[partial_count(1)])
      +- *(2) HashAggregate(keys=[Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16], functions=[])
         +- Exchange hashpartitioning(Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16, 5), ENSURE_REQUIREMENTS, [id=#289]
            +- *(1) HashAggregate(keys=[Cylinders#18, knownfloatingpointnormalized(normalizenanandzero(Horsepower#20)) AS Horsepower#20, Origin#24, Weight#21, Model#23, knownfloatingpointnormalized(normalizenanandzero(MPG#17)) AS MPG#17, knownfloatingpointnormalized(normalizenanandzero(Acceleration#22)) AS Acceleration#22, knownfloatingpointnormalized(normalizenanandzero(Displacement#19)) AS Displacement#19, Car#16], function

In [18]:
# obtener los carros con mayor MPG de origen japones

query_japan_cars = """
                  SELECT Car FROM cars
                  WHERE Origin = 'Japan'
                  ORDER BY MPG DESC
                  LIMIT 5
                  """
df_japan_cars_best_mpg = spark.sql(query_japan_cars)
df_japan_cars_best_mpg.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[MPG#17 DESC NULLS LAST], output=[Car#16])
+- *(2) HashAggregate(keys=[Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16], functions=[])
   +- Exchange hashpartitioning(Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16, 5), ENSURE_REQUIREMENTS, [id=#449]
      +- *(1) HashAggregate(keys=[Cylinders#18, knownfloatingpointnormalized(normalizenanandzero(Horsepower#20)) AS Horsepower#20, Origin#24, Weight#21, Model#23, knownfloatingpointnormalized(normalizenanandzero(MPG#17)) AS MPG#17, knownfloatingpointnormalized(normalizenanandzero(Acceleration#22)) AS Acceleration#22, knownfloatingpointnormalized(normalizenanandzero(Displacement#19)) AS Displacement#19, Car#16], functions=[])
         +- *(1) Filter ((isnotnull(Origin#24) AND AtLeastNNulls(n, Car#16,MPG#17,Cylinders#18,Displacement#19,Horsepower#20,Weight#21,Ac

In [19]:
# misma consulta con operadores PySpark

df_japan_cars_best_mpg2 = df_clean_sort.filter(df_clean_sort.Origin == 'Japan')\
                                       .orderBy(df_clean_sort.MPG, ascending=False)\
                                       .select('Car')\
                                       .limit(5)

df_japan_cars_best_mpg2.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[MPG#17 DESC NULLS LAST], output=[Car#16])
+- *(2) HashAggregate(keys=[Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16], functions=[])
   +- Exchange hashpartitioning(Cylinders#18, Horsepower#20, Origin#24, Weight#21, Model#23, MPG#17, Acceleration#22, Displacement#19, Car#16, 5), ENSURE_REQUIREMENTS, [id=#481]
      +- *(1) HashAggregate(keys=[Cylinders#18, knownfloatingpointnormalized(normalizenanandzero(Horsepower#20)) AS Horsepower#20, Origin#24, Weight#21, Model#23, knownfloatingpointnormalized(normalizenanandzero(MPG#17)) AS MPG#17, knownfloatingpointnormalized(normalizenanandzero(Acceleration#22)) AS Acceleration#22, knownfloatingpointnormalized(normalizenanandzero(Displacement#19)) AS Displacement#19, Car#16], functions=[])
         +- *(1) Filter ((isnotnull(Origin#24) AND AtLeastNNulls(n, Car#16,MPG#17,Cylinders#18,Displacement#19,Horsepower#20,Weight#21,Ac

In [20]:
df_japan_cars_best_mpg.show(5)
df_japan_cars_best_mpg2.show(5)

+-------------------+
|                Car|
+-------------------+
|          Mazda GLC|
|Honda Civic 1500 gl|
|         Datsun 210|
|     Datsun B210 GX|
|     Toyota Starlet|
+-------------------+

+-------------------+
|                Car|
+-------------------+
|          Mazda GLC|
|Honda Civic 1500 gl|
|         Datsun 210|
|     Datsun B210 GX|
|     Toyota Starlet|
+-------------------+



# Aprendizaje máquina

Dentro de la Inteligencia Artificial, el aprendizaje máquina (Machine Learning o ML) se enfoca en el estudio y diseño de algoritmos que permiten procesar una serie de datos para encontrar tendencias, patrones ocultos en los datos, construyendo modelos de aprendizaje. Existen diferentes tipos de aprendizaje, como lo es el parendizaje supervisado, no supervisado o por refuerzo.

El aprendizaje supervisado se parte de una colección de datos previamente etiquetados (significado semántico), denominado valor de clase, a partir de los cuales se trata de construir modelos que puedan predecir el valor de clase a partir de una nueva instancia no etiquetada. Los valores de clase pueden ser de dos tipos, discretos y continuos, lo que deriva en los dos tipos de aprendizaje supervisado: clasificación y regresión.

Por otro lado, en el parendizaje no supervisado se parte de un conjunto de datos no etiquetados (se desconoce el valor de clase de las instancias). Por lo tanto, en lugar de construir modelos para predevir un valor de clase, el aprendizaje no supervisado construye modelos que ayudan a entender la estructura de los datos en un hiper-espacio (nubes de datos o clusters). Este tipo de aprendizaje es muy útil para tareas como la identificación de valores atípicos (outliers), o como etapa previa (preprocesamiento) del aprendizaje supervisado (a los grupos identificados se les asigna un valor de clase).

#### Aprendizaje supervisado

Para el ejemplo, se estará usaudon la base de datos de BostonHousing ([link](https://www.kaggle.com/datasets/altavish/boston-housing-dataset/data))

In [21]:
path_file = '/content/drive/MyDrive/BigData_Maestria/Clase/BostonHousing.csv'
df = spark.read.csv(path_file, header=True, sep=",", inferSchema=True)
df.show(5)
print("Número de registros: " + str(df.count()))
print("Número de columnas: " + str(len(df.columns)))

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows

Número de registros: 506
Número de columnas: 14


In [22]:
df.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [23]:
# Limpieza básica de datos
#Se eliminan registros con valores nulos
df_clean = df.dropna()

#Se eliminan registros duplicados
df_clean= df_clean.dropDuplicates()

df_clean.explain()
df_clean.show(5)

print("Número de registros: " + str(df_clean.count()))
print("Número de columnas: " + str(len(df_clean.columns)))

== Physical Plan ==
*(2) HashAggregate(keys=[rad#1407, tax#1408, chas#1402, medv#1412, rm#1404, age#1405, ptratio#1409, black#1410, nox#1403, crim#1399, indus#1401, lstat#1411, dis#1406, zn#1400], functions=[])
+- Exchange hashpartitioning(rad#1407, tax#1408, chas#1402, medv#1412, rm#1404, age#1405, ptratio#1409, black#1410, nox#1403, crim#1399, indus#1401, lstat#1411, dis#1406, zn#1400, 5), ENSURE_REQUIREMENTS, [id=#618]
   +- *(1) HashAggregate(keys=[rad#1407, tax#1408, chas#1402, knownfloatingpointnormalized(normalizenanandzero(medv#1412)) AS medv#1412, knownfloatingpointnormalized(normalizenanandzero(rm#1404)) AS rm#1404, knownfloatingpointnormalized(normalizenanandzero(age#1405)) AS age#1405, knownfloatingpointnormalized(normalizenanandzero(ptratio#1409)) AS ptratio#1409, knownfloatingpointnormalized(normalizenanandzero(black#1410)) AS black#1410, knownfloatingpointnormalized(normalizenanandzero(nox#1403)) AS nox#1403, knownfloatingpointnormalized(normalizenanandzero(crim#1399)) A

El conjunto de datos de entrada, se tiene que dividir en subconjuntos para el proceso de entrenamiento: train, validation, test. En muchos casos, el particionamiento se realiza generando únicamente el train y test, usando para ello porcentajes de particionamiento típico 80-20, 70-30, a partir de un muestreo aleatorio simple.

Sin embargo, es Big Data es crucial preparar cuidadosamente dichos conjuntos, ya que la construcción de modelos es computacionalmente cara. Por ello, es preferible usar una estrategia de muestreo que garantice la representatividad de los diversos comportamientos que tienen los datos, para evitar sesgos de representatividad en los conjuntos train y test.



In [24]:
# Particionamiento típico
# Se define el valor por default del número de ejecutores
spark.conf.set("spark.sql.shuffle.partitions", "200")
train_data,test_data = df_clean.randomSplit([0.8,0.2], seed = 42)
print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")

Existen 410 instancias en el conjunto train, y 96 en el conjunto test


In [25]:
# prompt: imprimir el plan de transformación al llamar randomSplit

train_data.explain()
test_data.explain()


== Physical Plan ==
*(2) Sample 0.0, 0.8, false, 42
+- *(2) Sort [crim#1399 ASC NULLS FIRST, zn#1400 ASC NULLS FIRST, indus#1401 ASC NULLS FIRST, chas#1402 ASC NULLS FIRST, nox#1403 ASC NULLS FIRST, rm#1404 ASC NULLS FIRST, age#1405 ASC NULLS FIRST, dis#1406 ASC NULLS FIRST, rad#1407 ASC NULLS FIRST, tax#1408 ASC NULLS FIRST, ptratio#1409 ASC NULLS FIRST, black#1410 ASC NULLS FIRST, lstat#1411 ASC NULLS FIRST, medv#1412 ASC NULLS FIRST], false, 0
   +- *(2) HashAggregate(keys=[rad#1407, tax#1408, chas#1402, medv#1412, rm#1404, age#1405, ptratio#1409, black#1410, nox#1403, crim#1399, indus#1401, lstat#1411, dis#1406, zn#1400], functions=[])
      +- Exchange hashpartitioning(rad#1407, tax#1408, chas#1402, medv#1412, rm#1404, age#1405, ptratio#1409, black#1410, nox#1403, crim#1399, indus#1401, lstat#1411, dis#1406, zn#1400, 200), ENSURE_REQUIREMENTS, [id=#827]
         +- *(1) HashAggregate(keys=[rad#1407, tax#1408, chas#1402, knownfloatingpointnormalized(normalizenanandzero(medv#1412)) 

In [26]:
# Se imprime algunos registros del train y test generado
train_data.show(5)
test_data.show(5)

+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|   nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+
|0.62976| 0.0| 8.14|   0| 0.538|5.949|61.8|4.7075|  4|307|   21.0| 396.9| 8.26|20.4|
|0.06899| 0.0|25.65|   0| 0.581| 5.87|69.7|2.2577|  2|188|   19.1|389.15|14.37|22.0|
|0.03578|20.0| 3.33|   0|0.4429| 7.82|64.5|4.6947|  5|216|   14.9|387.31| 3.76|45.4|
|0.03961| 0.0| 5.19|   0| 0.515|6.037|34.5|5.9853|  5|224|   20.2| 396.9| 8.01|21.1|
|0.05372| 0.0|13.92|   0| 0.437|6.549|51.0|5.9604|  4|289|   16.0|392.85| 7.39|27.1|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+

Pero, ¿qué sucede si cambiamos el número de ejecutores en nuestro clúster de Spark? El optimizador Catalyst determina la forma óptima de particionar tus datos en función de los recursos de tu clúster y el tamaño de tu conjunto de datos. Dado que los datos en un DataFrame de Spark están particionados por filas y cada trabajador realiza su división independientemente de los demás trabajadores, si los datos en las particiones cambian, entonces el resultado de la división (por randomSplit()) no será el mismo.

In [27]:
# Se define el valor por default del número de ejecutores
spark.conf.set("spark.sql.shuffle.partitions", "1")
train_data,test_data = df_clean.randomSplit([0.8,0.2], seed = 42)
print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")


Existen 430 instancias en el conjunto train, y 76 en el conjunto test


In [28]:
# Se imprime algunos registros del train y test generado
train_data.show(5)
test_data.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.00906|90.0| 2.97|   0|  0.4|7.088|20.8|7.3073|  1|285|   15.3|394.72| 7.85|32.2|
|0.01301|35.0| 1.52|   0|0.442|7.241|49.3|7.0379|  1|284|   15.5|394.74| 5.49|32.7|
|0.01311|90.0| 1.22|   0|0.403|7.249|21.9|8.6966|  5|226|   17.9|395.93| 4.81|35.4|
| 0.0136|75.0|  4.0|   0| 0.41|5.888|47.6|7.3197|  3|469|   21.1| 396.9| 14.8|18.9|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+--

En principio, si los conjuntos train y test son de calidad (representativos), el modelo que se construya a partir de estos datos debría ser idelamente "idéntico". Sin embargo, en la práctica suelen haber variaciones, lo que no es deseable si se esta trabajando con grándes volumenes de datos.

#### Ejemplo de muestreo estratificado

In [29]:
df_clean.describe()

summary,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506,506,506,506,506,506,506,506,506,506,506,506,506,506
mean,3.6135235573122535,11.363636363636363,11.136778656126504,0.0691699604743083,0.5546950592885372,6.284634387351787,68.57490118577078,3.795042687747034,9.549407114624506,408.2371541501976,18.455533596837967,356.67403162055257,12.653063241106723,22.532806324110698
stddev,8.601545105332491,23.32245299451514,6.860352940897589,0.2539940413404101,0.11587767566755584,0.7026171434153232,28.148861406903595,2.10571012662761,8.707259384239366,168.53711605495903,2.1649455237144455,91.29486438415782,7.141061511348571,9.197104087379815
min,0.00632,0.0,0.46,0,0.385,3.561,2.9,1.1296,1,187,12.6,0.32,1.73,5.0
max,88.9762,100.0,27.74,1,0.871,8.78,100.0,12.1265,24,711,22.0,396.9,37.97,50.0


In [30]:
print("Número de registros: " + str(df_clean.count()))
print("Número de columnas: " + str(len(df_clean.columns)))

Número de registros: 506
Número de columnas: 14


In [31]:
from pyspark.sql.functions import col
from pyspark.ml.feature import QuantileDiscretizer

# Se calcula el valor de discretización a partir del número de bins. El resultado se almacena en "medv_binned"
num_bins = 2
quantile_discretizer = QuantileDiscretizer(numBuckets=num_bins, inputCol="medv", outputCol="medv_binned")

# Aplicar transformación
df_clean_bin = quantile_discretizer.fit(df_clean).transform(df_clean)

df_clean_bin.show()
print("Número de registros: " + str(df_clean_bin.count()))
print("Número de columnas: " + str(len(df_clean_bin.columns)))

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+-----------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|medv|medv_binned|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+-----------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|        1.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|        1.0|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|        1.0|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|        1.0|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|        1.0|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|        1.0|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|        1.0|
|0.14455|12.5| 7.87|   0|0.524

In [32]:
# Se obtiene el número de instancias por bin
stratum_counts = df_clean_bin.groupBy("medv_binned").count().collect()
total_count = df_clean_bin.count()

print(stratum_counts)
print(total_count)

[Row(medv_binned=1.0, count=255), Row(medv_binned=0.0, count=251)]
506


In [33]:
# Se calcula la probabilidad del test de cada bin de acuerdo al porcentaje de division a usar (70 - 30)

stratum_fractions = {row["medv_binned"]: 0.3 * (row["count"] / total_count)
                     for row in stratum_counts}

print(stratum_fractions)

{1.0: 0.15118577075098813, 0.0: 0.14881422924901186}


In [34]:
# Se generan los conjuntos a partir de muestreo estratificado
test_data = df_clean_bin.sampleBy("medv_binned", fractions=stratum_fractions, seed=42)
train_data = df_clean_bin.exceptAll(test_data)

print(f"""Existen {train_data.count()} instancias en el conjunto train, y {test_data.count()} en el conjunto test""")

train_data.show(5)
test_data.show(5)

Existen 435 instancias en el conjunto train, y 71 en el conjunto test
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-----------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|medv_binned|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-----------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|        1.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|        1.0|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|        1.0|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|        1.0|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|        1.0|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-----------+
only showing top 5 rows

+-------+----+-----+----+-----+-----+----

#### Preparación del dataframe para ser procesado con algoritmos de ML en PySpark

Se usa VectorAssembler para generar una o más columnas en la cual, se "encapuslan" en un vector único los valores de los descriptores a usar en el proceso de aprendizaje.

In [35]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat'], outputCol = 'Attributes')
output_train = assembler.transform(train_data)
output_test = assembler.transform(test_data)

output_train.show(5)
output_test.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-----------+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|medv_binned|          Attributes|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+-----------+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|        1.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|        1.0|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|        1.0|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|        1.0|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|        1.0|[0.06905,0.0,2.18...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------

#### Ejemplo de Regresión Lineal

In [36]:
# Se manda a entrenar con un modelo de regresion Lineal
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(output_train)


In [37]:
# Se Imprimen los valores de los coeficientes
print ("The coefficient of the model is : ", lr_model.coefficients)
print ("The Intercept of the model is : ", lr_model.intercept)

The coefficient of the model is :  [-0.031034601210892566,0.01802274847148352,-0.022315059672006466,2.0062887531369724,-7.172130109899271,4.031876007066287,0.0,-0.7300192446759362,0.0,0.0,-0.761627024488328,0.007644093273434244,-0.564161432039539]
The Intercept of the model is :  22.531404042417087


In [38]:
# Se aplica el modelo al conjunto test
Pred_lr = lr_model.evaluate(output_test)

In [39]:
# Evaluación del modelo
from pyspark.ml.evaluation import RegressionEvaluator

#Root Mean Square Error
eval_lr = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse_lr = eval_lr.evaluate(Pred_lr.predictions)
print("RMSE: %.3f" % rmse_lr)

# Mean Square Error
mse = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval_lr.evaluate(Pred_lr.predictions, {eval_lr.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 4.786
MSE: 22.906
MAE: 3.614
r2: 0.638


In [60]:
# salvando el modelo lr_model y si existe sobreescribirlo
lr_model.save("/content/drive/MyDrive/BigData_Maestria/Clase/lr_model")
# Leer el modelo del directorio
from pyspark.ml.regression import LinearRegressionModel
lr_model_loaded = LinearRegressionModel.load("/content/drive/MyDrive/BigData_Maestria/Clase/lr_model")